# Limpieza y Arreglo de Datos 

En este cuaderno se detalla el proceso de limpieza y arreglo de los dos conjuntos de datos: *products.csv* y *products_categories.csv*. El objetivo es preparar los datasets, para poder manejarlos posteriormente sin errores.

## Índice 

1. Importación paquetes
2. Carga de Datos
3. Categorías
4. Productos
5. Nombres
6. Conclusiones

# Importación de paquetes

Como siempre importamos los paquetes necesarios para realizar la limpieza

In [149]:
import pandas as pd
import re
import spacy
from collections import Counter, OrderedDict
# Semilla para realizar experimentos reproducibles
seed = 125

# Carga de Datos

Cargamos en memoria los dos ficheros y comprobamos que lo hayan hecho correctamente.

In [150]:
df_products = pd.read_csv('../Data/products.csv')
df_products.sample(5,random_state=seed)

,product_id,sku,name,marca_value,short_description,analytic_category,picture
23485,76086,MISUPI,SuperPack personalizado meu Pipo,Mi Pipo,<p>El SuperPack personalizado de Mi Pipo es id...,NaN,https://www.mifarma.es/media/catalog/product/m...
27586,74774,41401,2x2ml soro & camaleão de cor mágica,Camaleon,<p>Si tu piel necesita un efecto lifting inmed...,Cosmética y Belleza,https://www.mifarma.es/media/catalog/product/4...
2777,4388,152307,Aquilea Gases 60 Comprimidos,Aquilea,<p>Solución natural a los gases gracias a su c...,Herbolario,https://www.mifarma.es/media/catalog/product/1...
12655,79092,BN013,Colageno Marino + Silicio Organico Binature 18...,Binature,<p>Complemento alimenticio con Colágeno marino...,Herbolario,https://www.mifarma.es/media/catalog/product/b...
12783,61316,44701381,Tommee Tippee Easy Drink Taza Aprendizaje 230m...,Tommee tippee,<p>Taza con pajita fabricada en suave silicona...,Infantil,https://www.mifarma.es/media/catalog/product/4...


In [151]:
df_categories = pd.read_csv('../Data/products_categories.csv')
df_categories.sample(5,random_state=seed)

,sku,cat1,cat2,cat3
4038,177366.gris,Infantil,Biberones,Biberones
1305,153978,Salud,Problemas cardiovasculares y circulación,Piernas cansadas y varices
5430,187507,Salud,Ortopedia,Plantillas
4805,183139,Cosmética y Belleza,Ojos,Contorno de Ojos
2870,168461,Infantil,Higiene infantil,Crema pañal


# Categorías

Empezaremos con el dataset de categorías, ya que es el *"menos"* problemático y también porque nos servirá de ayuda para completar datos en el otro dataframe.

Si recordamos del Análisis Exploratorio, vimos que hay ciertos número de valores nulos en las subcategorías(**cat2** y **cat3**). Así que sustituiremos esos valores nulos por un simple guión que indique que carece de tal categoría, ya que carecemos de fuentes de datos que nos puedan dar indicios sobre la subcategoría del producto

In [152]:
df_categories.isna().sum()

sku       0
cat1      0
cat2     20
cat3    300
dtype: int64

In [153]:
df_categories.fillna('-', inplace=True)
df_categories.isna().sum()

sku     0
cat1    0
cat2    0
cat3    0
dtype: int64

Lo siguiente será estandarizar todos los campos, ya que se identificaron *"erratas"* a la hora de imputar los nombres. Para ello, se eliminan los espacios innecesarios e inonsitencias de mayúsculas y minúsculas.

In [154]:
df_categories.loc[:, ["cat1", "cat2", "cat3"]] = df_categories[["cat1", "cat2", "cat3"]].apply(lambda r: [str(n).strip().capitalize() for n in r], axis=1, result_type="broadcast")
df_categories

,sku,cat1,cat2,cat3
0,00.01.10.014,Cosmética y belleza,Corporal,Hidratación
1,00.071697.000.000,Infantil,Juguetes,Mordedores
2,000133,Infantil,Higiene infantil,Canastillas y kits bebé
3,000147,Higiene y cuidado personal,Facial,Desmaquillantes y limpiadores
4,000148,Cosmética y belleza,Manos,Crema de manos
...,...,...,...,...
11154,ZTV15215,Higiene y cuidado personal,Íntima,Limpieza
11155,ZTV15216,Higiene y cuidado personal,Cabello,Champu
11156,zuecoloki,Salud,Ortopedia,Zapatos y zuecos
11157,zuecooden,Salud,Ortopedia,Zapatos y zuecos


Finalmente, dado que no se encontraron más inconvenientes con estos datos, se guardan en un nuevo fichero que será el que se use a partir de ahora.

In [155]:
df_categories.to_csv('./clean_categories.csv',index=False)

# Productos

Este dataset presenta varias *"trabas"*, así que veamos como darles solución. El primer gran incoveniente es que había un gran porcentaje de productos sin **analytic_category**. Viendo que los valores de la columna **cat1** coinciden con las de esta, se intentará rescatar su valores a partir de la clave **sku** 

In [156]:
df_products.isna().sum() #Cantidad de valores que faltan

product_id              0
sku                     0
name                   14
marca_value            32
short_description      23
analytic_category    3537
picture                 6
dtype: int64

Antes de realizar el *merge* comprobamos que los nombres de categorías sean exactamente iguales, para que no aparezcan categorías duplicadas.

In [157]:
df_categories['cat1'].value_counts()

Cosmética y belleza           3367
Higiene y cuidado personal    2994
Salud                         2855
Infantil                      1304
Nutrición                      619
Veterinaria                     20
Name: cat1, dtype: int64

In [158]:
df_products['analytic_category'].value_counts()

Cosmética y Belleza    8336
Higiene                5410
Infantil               4901
Herbolario             2801
Nutrición              1600
Ortopedia               572
Vida Íntima             340
Óptica                  244
Perfumeria              165
Veterinaria              47
Name: analytic_category, dtype: int64

Vemos que hay algunas categorías que llevarían a errores. Vamos a prevenirlos.

In [159]:
df_categories.loc[df_categories['cat1'] == 'Higiene y cuidado personal', 'cat1'] = 'Higiene'
df_categories.loc[df_categories['cat1'] == 'Cosmética y belleza', 'cat1'] = 'Cosmética y Belleza'

In [160]:
recoverable = df_products.loc[df_products["analytic_category"].isna(), ["sku"]].reset_index().merge(df_categories[["sku", "cat1"]], on="sku")

In [161]:
# Comprobamos que se ha ejecutado correctamente
df_products.loc[recoverable["index"], "analytic_category"]

5        NaN
20641    NaN
6        NaN
11727    NaN
17477    NaN
        ... 
27376    NaN
27466    NaN
27493    NaN
27754    NaN
27800    NaN
Name: analytic_category, Length: 868, dtype: object

In [162]:
# Asignamos los valores recuperados
df_products.loc[recoverable["index"], "analytic_category"] = recoverable["cat1"].values

In [163]:
# Comprobamos que ha funcionado correctamente
df_products.loc[recoverable["index"].values, "analytic_category"]

5                   Infantil
20641               Infantil
6                   Infantil
11727               Infantil
17477               Infantil
                ...         
27376                Higiene
27466    Cosmética y Belleza
27493    Cosmética y Belleza
27754                Higiene
27800                Higiene
Name: analytic_category, Length: 868, dtype: object

In [164]:
df_products.isna().sum()

product_id              0
sku                     0
name                   14
marca_value            32
short_description      23
analytic_category    2669
picture                 6
dtype: int64

Conseguimos reducir sustancialmente los valores perdidos de categorías de cerca de *3500* pasan a alrededor de *2700*. Aun así sigue siendo un recuento bastante grande respecto al total del conjunto de datos.

**IDEA FELIZ**. Por lo general una marca se suele enmarcar dentro de una misma categoría de productos, ¿por qué no restaurar los valores que quedan de la columna **analytic_category** a través de las categorías de las marcas?

In [165]:
df_marcas = df_products[['marca_value','analytic_category']].drop_duplicates(ignore_index=True)
df_marcas.sample(5,random_state=seed)

,marca_value,analytic_category
1690,Nestle Nan,NaN
1193,Sandoz,Cosmética y Belleza
219,Fluocaril,Higiene
416,Disney,Infantil
1648,Cerave,Cosmética y Belleza


Eliminamos aquellas filas que tengan nula el campo de categoría, ya que no son válidas si queremos recuperar los valores perdidos.

In [166]:
df_marcas.shape

(1913, 2)

In [167]:
df_marcas.dropna(inplace=True)

In [168]:
df_marcas.shape

(1566, 2)

Agrupamos las marcas y nos quedamos con las que se asocian solamente a una única categoría, ya que de otro modo no se puede adivinar a que categoría pertenece el producto de la marca.

In [169]:
df_marcas_count = df_marcas.groupby(['marca_value'],as_index=False).agg(count=('analytic_category','count'),cat=('analytic_category','first'))

In [170]:
df_marcas_count = df_marcas_count[df_marcas_count['count'] == 1]
df_marcas_count.sample(5,random_state=seed)

,marca_value,count,cat
749,Yammy,1,Infantil
672,Sophie La Girafe,1,Infantil
509,Olioseptil,1,Herbolario
483,Nickelodeon,1,Higiene
263,Fisiopharma,1,Herbolario


Recuperamos los valores perdidos en el dataframe de productos

In [171]:
recoverable = df_marcas_count[["marca_value", "cat"]].merge(df_products.loc[df_products["analytic_category"].isna(), ["marca_value"]].reset_index(), on="marca_value")
recoverable

,marca_value,cat,index
0,4moms,Infantil,26751
1,Adisan,Infantil,9170
2,AeroChamber,Ortopedia,16245
3,Almiron,Infantil,11629
4,Ausonia,Higiene,18243
...,...,...,...
629,Warmies,Infantil,15630
630,Warmies,Infantil,21320
631,Warmies,Infantil,22076
632,Warmies,Infantil,26475


In [172]:
df_products.loc[recoverable["index"], "analytic_category"] = recoverable["cat"].values

In [173]:
df_products.shape

(27953, 7)

In [174]:
df_products.isna().sum()

product_id              0
sku                     0
name                   14
marca_value            32
short_description      23
analytic_category    2035
picture                 6
dtype: int64

Se ha conseguido reducir de nuevo el recuento de categorías perdidas, pero sigue siendo un número cuantioso.

**IDEA FELIZ** Dado que tenemos las descripciones de los productos, podemos crear un diccionario con palabras clave que se suelen repetir. De esta manera puede que consigamos imputar más valores perdidos, sin perder rigor en los datos.

Dado que las descripciones tienen etiquetas de HTML, estas deben ser eliminadas si queremos tokenizar correctamente los textos.

In [175]:
df_products.loc[:,('short_description')] = df_products['short_description'].str.replace(r'<[^<>]*>', '', regex=True)
df_products.sample(5,random_state=seed)

,product_id,sku,name,marca_value,short_description,analytic_category,picture
23485,76086,MISUPI,SuperPack personalizado meu Pipo,Mi Pipo,El SuperPack personalizado de Mi Pipo es ideal...,Infantil,https://www.mifarma.es/media/catalog/product/m...
27586,74774,41401,2x2ml soro & camaleão de cor mágica,Camaleon,Si tu piel necesita un efecto lifting inmediat...,Cosmética y Belleza,https://www.mifarma.es/media/catalog/product/4...
2777,4388,152307,Aquilea Gases 60 Comprimidos,Aquilea,Solución natural a los gases gracias a su comb...,Herbolario,https://www.mifarma.es/media/catalog/product/1...
12655,79092,BN013,Colageno Marino + Silicio Organico Binature 18...,Binature,Complemento alimenticio con Colágeno marino hi...,Herbolario,https://www.mifarma.es/media/catalog/product/b...
12783,61316,44701381,Tommee Tippee Easy Drink Taza Aprendizaje 230m...,Tommee tippee,Taza con pajita fabricada en suave silicona id...,Infantil,https://www.mifarma.es/media/catalog/product/4...


Por otro lado, también se vio la presencia de carácteres de salto de línea. Estos deben ser eliminados, así como normalizar las palabras para que las mayúsculas no afecten a la hora de crear el diccionario de palabras.

In [176]:
df_products.loc[:,'short_description'] = df_products['short_description'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value='', regex=True)
df_products.loc[:,'short_description'] = df_products['short_description'].str.lower()
df_products['short_description'].fillna('-',inplace=True)
df_products.sample(5,random_state=seed)

,product_id,sku,name,marca_value,short_description,analytic_category,picture
23485,76086,MISUPI,SuperPack personalizado meu Pipo,Mi Pipo,el superpack personalizado de mi pipo es ideal...,Infantil,https://www.mifarma.es/media/catalog/product/m...
27586,74774,41401,2x2ml soro & camaleão de cor mágica,Camaleon,si tu piel necesita un efecto lifting inmediat...,Cosmética y Belleza,https://www.mifarma.es/media/catalog/product/4...
2777,4388,152307,Aquilea Gases 60 Comprimidos,Aquilea,solución natural a los gases gracias a su comb...,Herbolario,https://www.mifarma.es/media/catalog/product/1...
12655,79092,BN013,Colageno Marino + Silicio Organico Binature 18...,Binature,complemento alimenticio con colágeno marino hi...,Herbolario,https://www.mifarma.es/media/catalog/product/b...
12783,61316,44701381,Tommee Tippee Easy Drink Taza Aprendizaje 230m...,Tommee tippee,taza con pajita fabricada en suave silicona id...,Infantil,https://www.mifarma.es/media/catalog/product/4...


Nos quedamos con las descripciones y sus índices para que sean más fácil después realizar el update del dataframe de productos

In [177]:
descriptions = df_products[df_products['analytic_category'].isna()]['short_description'].to_list()
indexes = df_products[df_products['analytic_category'].isna()]['short_description'].index
descriptions[:10]

['pack de cuidados solares para una óptima protección facial y corporal frente a las radiaciones solares. ofrecen una alta protección, al tiempo que protegen las células del fotoenvejecimiento y activan las defensas naturales de la piel. indicados especialmente para el cuidado de las pieles muy claras.',
 'mussvital gel de baño sin jabón es una fórmula específica para la higiene y el cuidado diario de la piel sensible en adultos y niños.  cuida y protege eficazmente la piel más delicada, limpiando sin irritar y respetando la barrera hidrolipídica. ahora, gratis 1 loción hidratante y 1 gel, ambos en formato de 100ml.- sin parabenos.',
 'lucha de forma activa contra la caída del cabello, estimula y fortalece el cabello y las uñas. ideal en épocas de cambios hormonales o en la dieta. se puede consumir durante el período de lactancia.',
 'suplemento alimenticio rico en vitamina c. ayuda a proteger la salud general y en particular el sistema inmunitario. apto para veganos, vegetarianos y ko

Se ha hecho uso de un paquete especial para problemas de procesamiento de lenguajes para tokenizar el texto.

In [178]:
# Se carga el paquete de español
nlp = spacy.load("es_core_news_sm")

# Juntamos en un texto todas las descripciones
comb_desc = ' '.join(descriptions)

doc = nlp(comb_desc)

# Extraemos las palabras relevantes y contamos su aparición en el doc
words = [token.text for token in doc if not token.is_stop and not token.is_punct]
word_freq = Counter(words)

Nos hemos quedado con las **150** palabras más relevantes, de las cuales hemos extraído las que hemos considerado menos ambiguas. Con esto nos referimos a palabras que unívocamente podríamos relacionar a un tipo de categoría.

In [179]:
common_words = word_freq.most_common(150)
print (common_words)

[('y', 1566), ('a', 667), ('piel', 276), ('bebé', 238), ('pack', 218), ('chupete', 207), ('tetina', 172), ('ideal', 152), ('complemento', 125), ('bebés', 125), ('silicona', 124), ('alimenticio', 121), ('base', 117), ('crema', 116), ('suavinex', 115), ('forma', 110), ('diseño', 102), ('biberón', 101), ('pieles', 99), (' ', 96), ('gel', 94), ('meses', 93), ('suave', 91), ('o', 89), ('color', 87), ('agua', 83), ('corporal', 80), ('indicado', 80), ('gafas', 79), ('gracias', 76), ('regalo', 76), ('hidratante', 73), ('e', 72), ('sol', 71), ('cuidado', 68), ('natural', 68), ('bpa', 67), ('productos', 67), ('niños', 66), ('pequeños', 64), ('divertido', 63), ('especialmente', 62), ('contiene', 62), ('protege', 61), ('facial', 60), ('mordedor', 60), ('cabello', 59), ('manos', 58), ('+', 58), ('aceite', 57), ('higiene', 55), ('fácil', 55), ('parabenos', 53), ('apto', 52), ('3', 52), ('chicco', 52), ('2', 51), ('anatómica', 49), ('látex', 49), ('protección', 48), ('1', 48), ('chupetes', 47), ('hid

In [180]:
# Creamos nuestro diccionario de palabras en orden de frecuencia para evitar solapamientos
key_words = ["bebé","chupete","tetina","bebés","suavinex","biberón","niños","gafas","higiene","cabello","chicco","encías","chupetes","rostro","baño","vitaminas","jabón","infantil","champú","gafa","perfume","mam"]
value_cat = ["Infantil","Infantil","Infantil","Infantil","Infantil","Infantil","Infantil","Óptica","Higiene","Cosmética y Belleza","Infantil","Higiene","Infantil","Cosmética y Belleza","Higiene","Nutrición","Higiene","Infantil","Higiene","Óptica","Perfumeria","Infantil"]
cat_dict = OrderedDict(zip(key_words,value_cat))
print(f'Nº palabras clave: {len(key_words)}')
print(f'Categoría más repetida: {Counter(value_cat).most_common(1)}')

Nº palabras clave: 22
Categoría más repetida: [('Infantil', 11)]


In [181]:
# Función para asignar la categoría dependiendo de la descripcion
def word_in_catalog(desc,catalog):
    for k,w in catalog.items():
        if k in desc:
            return w

In [182]:
for i in indexes:
    df_products.loc[i,'analytic_category'] = word_in_catalog(df_products.loc[i,'short_description'],cat_dict)

In [183]:
df_products.isna().sum()

product_id              0
sku                     0
name                   14
marca_value            32
short_description       0
analytic_category    1061
picture                 6
dtype: int64

Con esto hemos logrado rebajar en *otros mil* las categorías perdidas, algo verdaderamente impresionante con algo tan simple. Podríamos seguir intentando recuperar valores a partir de las descripciones con algún tipo de modelo predictivo o viendo el contexto de las palabras, pero ello requiere mucho tiempo del que no se dispone. 

Dicho esto, el resto de nulos que permanecen serán eliminados, porque ya no suponen un gran porcentaje de valores perdidos.

In [184]:
percent_missing = df_products.isnull().sum() * 100 / df_products.shape[0]
df_missing_values = pd.DataFrame({ 
    'column_name': df_products.columns,
    'percent_missing': percent_missing
})
df_missing_values.sort_values(by='percent_missing',ascending=False,inplace=True)
print(df_missing_values.to_string(index=False))

      column_name  percent_missing
analytic_category         3.795657
      marca_value         0.114478
             name         0.050084
          picture         0.021465
       product_id         0.000000
              sku         0.000000
short_description         0.000000


Se ha pasado de **12%** a un **4%** lo cual consideramos un número razonable. En cuanto al resto de variables con valores perdidos, la de *marca* es la única que podemos salvar algunos registros, dado que al echar un vistazo nos hemos percatado de que algunos nombres de producto contiene también el de la marca.

In [185]:
df_products[df_products['marca_value'].isna()].sample(10,random_state=seed)

,product_id,sku,name,marca_value,short_description,analytic_category,picture
16104,33681,CONFETTI.L.CIR,Sujetador de Lactancia Medela Bravado Confetti...,NaN,"las copas no incluyen aro ni costuras, mejoran...",Infantil,https://www.mifarma.es/media/catalog/product/c...
12641,11609,DUVIT,Dr Dunner Multivital Complemento Alimenticio 4...,NaN,complemento alimenticio con 13 vitaminas y 11 ...,Nutrición,https://www.mifarma.es/media/catalog/product/d...
21037,23902,DUEQUECO,Dr Dunner Echinol Bio Equinacea 80 Comprimidos,NaN,dr dunner echinol bio equinacea 80 comprimidos.,Herbolario,https://www.mifarma.es/media/catalog/product/d...
5939,23884,DUCAR01,Dr Dunner Silygold Cardo Mariano 80 Comprimidos,NaN,dr dunner silygold cardo mariano 80 comprimidos.,Herbolario,https://www.mifarma.es/media/catalog/product/d...
923,7105,157899,Dr Dunner Cinnulin 40 Capsulas,NaN,cuida tus niveles de azúcar: complemento alime...,Herbolario,https://www.mifarma.es/media/catalog/product/1...
5738,23899,DUGIN,Dr Dunner Ginkocel Biloba 40 Comprimidos,NaN,dr dunner gastrodiet 40 comprimidos.,Herbolario,https://www.mifarma.es/media/catalog/product/d...
13094,78484,152937.0,Sublime Curl Champu Sublimador Rizos Rene Furt...,NaN,sublime curl champu sublimador rizos rene furt...,Cosmética y Belleza,https://www.mifarma.es/media/catalog/product/1...
4451,513,317171x2-317171-317171,La Roche Posay Solucion Micelar 200ml + 200 ml...,NaN,solucion micelar la roche posay desmaquillante...,None,https://www.mifarma.es/media/catalog/product/3...
9188,17205,DUGAS,Dr Dunner Gastrodiet 40 Comprimidos,NaN,reduce los problemas intestinales y elimina lo...,Herbolario,https://www.mifarma.es/media/catalog/product/d...
20926,1031,SERV262253X2-262253-262253,Champú Revitalizante Interapothek 400ml Duplo,NaN,champú formulado con provitamina b5 especialme...,Higiene,https://www.mifarma.es/media/catalog/product/s...


In [186]:
# Recuperamos los nombres de productos
product_names = df_products[df_products['marca_value'].isna()]['name'].to_list()
indexes = df_products[df_products['marca_value'].isna()]['name'].index
product_names[:20]

['Prelox 60 Comprimidos',
 'Dr Dunner Cinnulin 40 Capsulas',
 'Interapothek Champu Anticaida 400 ml + 400 ml DUPLO',
 'Dr Dunner Menosoy 60 Capsulas',
 'La Roche Posay Solucion Micelar 200ml + 200 ml DUPLO',
 'Medela Bravado Design Seamless Latte M',
 'Dr Dunner Ginkocel Biloba 40 Comprimidos',
 'Dr Dunner Pagosid 80 Comprimidos',
 'Dr Dunner Silygold Cardo Mariano 80 Comprimidos',
 'Medela Bravado Design Seamless Latte L',
 'Dr Dunner Gastrodiet 40 Comprimidos',
 'Sujetador de Lactancia Medela Bravado Confetti Nursing Bra Talla M Ciruela ',
 'Medela Bravado Design Bliss Negro 100 D/E DD',
 'Medela Discos Absorbentes Desechables 30 Discos + 30 Discos DUPLO',
 'Dr Dunner Derinol 40 Capsulas',
 'Dr Dunner Nattolin Osteo 30 Capsulas',
 'Dr Dunner Multivital Complemento Alimenticio 40 Comprimidos',
 'Sublime Curl Champu Sublimador Rizos Rene Furterer 250ml',
 'Lierac Magnificence Crema Aterciopelada 50ml + Serum Antiarrugas 30ml + Contorno de Ojos de REGALO',
 'Sujetador de Lactancia Medel

In [187]:
# Juntamos en un texto todas las descripciones
prods = ' '.join(product_names)

doc = nlp(prods)

# Extraemos las palabras relevantes y contamos su aparición en el doc
words = [token.text for token in doc if not token.is_stop and not token.is_punct]
word_freq = Counter(words)

In [188]:
common_words = word_freq.most_common()
print (common_words)

[('Dr', 12), ('Dunner', 12), ('Medela', 9), ('Comprimidos', 7), ('Bravado', 7), ('40', 5), ('Capsulas', 5), ('+', 5), (' ', 5), ('ml', 4), ('Sujetador', 4), ('Lactancia', 4), ('Confetti', 4), ('Nursing', 4), ('Bra', 4), ('Talla', 4), ('Ciruela', 4), ('30', 4), ('60', 3), ('DUPLO', 3), ('Design', 3), ('80', 3), ('L', 3), ('Discos', 3), ('Interapothek', 2), ('Champu', 2), ('400', 2), ('Seamless', 2), ('Latte', 2), ('M', 2), ('250ml', 2), ('MiRebotica', 2), ('Gel', 2), ('Efecto', 2), ('Prelox', 1), ('Cinnulin', 1), ('Anticaida', 1), ('Menosoy', 1), ('Roche', 1), ('Posay', 1), ('Solucion', 1), ('Micelar', 1), ('200ml', 1), ('200', 1), ('Ginkocel', 1), ('Biloba', 1), ('Pagosid', 1), ('Silygold', 1), ('Cardo', 1), ('Mariano', 1), ('Gastrodiet', 1), ('Bliss', 1), ('Negro', 1), ('100', 1), ('D', 1), ('E', 1), ('DD', 1), ('Absorbentes', 1), ('Desechables', 1), ('Derinol', 1), ('Nattolin', 1), ('Osteo', 1), ('Multivital', 1), ('Complemento', 1), ('Alimenticio', 1), ('Sublime', 1), ('Curl', 1), (

In [189]:
# Se ha hecho una búsqueda en Internet para comrpobar que existen
key_words = ['Dunner', 'Medela', 'Interapothek', 'MiRebotica', 'Rene', 'Furterer', 'Lierac', 'Roche', 'Posay', 'Farlane']
marcas_value = ['Dr Dunner', 'Medela', 'Interapothek', 'MiRebotica', 'Rene Furterer', 'Rene Furterer', 'Lierac','La Roche Posay', 'La Roche Posay', 'Farline']

In [190]:
# Comprobamos que existan esas marcas existan dentro de nuestro catálogo
df_products[df_products['marca_value'].isin(marcas_value)]['marca_value'].value_counts()

La Roche Posay    364
Rene Furterer     178
Lierac            155
Medela            114
Farline           108
Name: marca_value, dtype: int64

Vemos que algunas marcas no aparecen, esto puede deberse a que estén escritas de otra manera. Comprobémoslo! 

In [191]:
df_products.loc[df_products['marca_value'].str.contains('Dr', na=False),'marca_value'].value_counts()

DrBrown´s       158
Drasanvi        145
Dr. Hauschka    109
Dr. Scholl       83
Dr. Tree          7
Dr. Dunner        3
Name: marca_value, dtype: int64

In [192]:
df_products.loc[df_products['marca_value'].str.contains('Inter', na=False),'marca_value'].value_counts()

InterApothek          322
Inter-Pharma           53
Dietéticos Intersa      4
Name: marca_value, dtype: int64

In [193]:
df_products.loc[df_products['marca_value'].str.contains('Mi', na=False),'marca_value'].value_counts()

Mi Pipo          76
Mia Laurens      75
Mi Rebotica      61
Mifarma Daily    20
Mifarma          12
Mifarma Baby      8
Migrasin          8
Milton            4
Mitigal           4
Miradent          1
Name: marca_value, dtype: int64

Vemos que efectivamente se trataba de eso, por tanto cambiamos las palabras clave

In [194]:
marcas_value = ['Dr. Dunner', 'Medela', 'InterApothek', 'Mi Rebotica', 'Rene Furterer', 'Rene Furterer', 'La Roche Posay', 'La Roche Posay', 'Farline']
marcas_dict = OrderedDict(zip(key_words,marcas_value))

In [195]:
for i in indexes:
    df_products.loc[i,'marca_value'] = word_in_catalog(df_products.loc[i,'name'],marcas_dict)

In [196]:
df_products.isna().sum()

product_id              0
sku                     0
name                   14
marca_value             4
short_description       0
analytic_category    1061
picture                 6
dtype: int64

Se ha reducido considerablemente el número de marcas que faltan. Por tanto, ya podemos dar por finalizada la recuperación de valores y eliminaremos todas aquellas filas que contienen valores nulos.

In [197]:
df_products.dropna(how='any',inplace=True)

In [198]:
df_products.isna().sum()

product_id           0
sku                  0
name                 0
marca_value          0
short_description    0
analytic_category    0
picture              0
dtype: int64

Ya para finalizar, hemos considerado que la columna de *pictures* no aporta ningún valor, ya que los links no funcionan y además lo que pretendemos desarrollar posteriormente no implica el uso de estas supuestas imágenes.

In [199]:
df_products.drop(columns=['picture'],inplace=True)

## Nombres de productos


In [200]:
p_id = df_products[['product_id']].duplicated(keep='first').sum()
p_id_sku = df_products[['product_id','sku']].duplicated(keep='first').sum()
p_id_sku_n = df_products[['product_id','sku','name']].duplicated(keep='first').sum()
print(f'IDs de productos repetidos: {p_id}')
print(f'Combinación de IDs y SKUs de productos repetidos: {p_id_sku}')
print(f'Productos repetidos: {p_id_sku_n}')

IDs de productos repetidos: 4649
Combinación de IDs y SKUs de productos repetidos: 4609
Productos repetidos: 0


Normalizamos el nombre de los productos para ver si hay algun duplicado.

In [201]:
df_products.loc[:, 'name'] = df_products['name'].apply(lambda r: str(r).strip().lower())
df_products.sample(5,random_state=seed)

,product_id,sku,name,marca_value,short_description,analytic_category
10162,9211,SCF766/00=167969.Azul,avent vaso termico azul 12m+ scf766/00 260ml,Avent,vaso térmico para mantener el líquido a la tem...,Infantil
11494,44558,01012.monorosa,dr browns chupete ortodóntico perform talla 2:...,DrBrown´s,diseñado para la boca del bebe en crecimiento,Infantil
23670,7937,346526,carmex cherry jar spf15 hidratante labial 75mg,Carmex,bálsamo hidratante para labios cereza. alivio ...,Cosmética y Belleza
16341,3586,317735,polysianes gel crema autobronceador cara y cue...,Polysianes,polysianes gel crema autobronceador cara y cue...,Cosmética y Belleza
246,2147,391847,lacer pasta dental anticaries 125ml,Lacer,pasta dentífrica lacer indicada para la preven...,Higiene


In [202]:
df_products[df_products[['product_id','sku','name']].duplicated(keep='first')]

,product_id,sku,name,marca_value,short_description,analytic_category
15608,80567,005940,"melatonina con melisa, pasiflora y tila sotya ...",Sotya,suplemento alimenticio que ayuda a conciliar e...,Salud
16853,80564,005933,​melatonina complex sotya 550 mg 60 cápsulas,Sotya,complemento alimenticio que ayuda a conciliar ...,Salud
18769,5036,159115,sesderma c-vit liposomal serum 30 ml,Sesderma,sérum facial con una alta concentración de vit...,Cosmética y Belleza


Efectivamente vemos que hay algun duplicado, así que los eliminamos.

In [203]:
df_products = df_products.drop_duplicates(subset=['product_id','sku','name'] ,keep='first')

Al igual que el otros dataframe, lo guardamos para que pueda ser utilizado

In [204]:
df_products.to_csv('./clean_products.csv',index=False)

# Conclusiones

De todo este proceso sacamos en claro las siguientes conclusiones:

- En ambos datasets se han regularizado los campos de texto
- Se ha recuperado un **8%** de valores perdidos para el campo *analytic_category*
- Se han suprimido registros una vez que se ha reducido los nulos y necesitamos de técnicas mucho más complejas
- Se ha prescindido de la columna *picture* dado que no aporta valor 
- Se ha normalizado los nombre de productos